
# AI Music Generation System 🎵 (Markov Model Version)
 


### Project Description
This project builds an AI system capable of generating new music sequences 
using deep learning.

The system performs:
- Note sequence extraction
- Data preprocessing
- Sequence modeling using LSTM
- Music generation




In [2]:

import os
import glob
import random
import music21


In [13]:

dataset_path = r"C:\Users\amina\OneDrive\Desktop\midi dataset"
midi_files = glob.glob(os.path.join(dataset_path, "*.mid"))
print("Total MIDI files found:", len(midi_files))


Total MIDI files found: 26


In [16]:

notes = []

for file in midi_files:
    midi = music21.converter.parse(file)
    parts = music21.instrument.partitionByInstrument(midi)
    
    if parts:
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes
    
    for element in notes_to_parse:
        if isinstance(element, music21.note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, music21.chord.Chord):
            notes.append(".".join(str(n) for n in element.normalOrder))

print("Total notes extracted:", len(notes))


Total notes extracted: 9332


In [17]:

markov_chain = {}

for i in range(len(notes) - 1):
    current_note = notes[i]
    next_note = notes[i + 1]
    
    if current_note not in markov_chain:
        markov_chain[current_note] = []
    
    markov_chain[current_note].append(next_note)

print("Markov model created with", len(markov_chain), "states.")


Markov model created with 190 states.


In [18]:

def generate_music(length=120):
    start_note = random.choice(notes)
    generated = [start_note]
    
    for _ in range(length):
        current_note = generated[-1]
        
        if current_note in markov_chain:
            next_note = random.choice(markov_chain[current_note])
            generated.append(next_note)
        else:
            break
    
    return generated

generated_notes = generate_music()
print("Generated sequence length:", len(generated_notes))


Generated sequence length: 121


In [19]:

def save_midi(note_sequence, filename="generated_music.mid"):
    output_notes = []
    
    for pattern in note_sequence:
        if "." in pattern or pattern.isdigit():
            notes_in_chord = pattern.split(".")
            chord_notes = []
            for note in notes_in_chord:
                new_note = music21.note.Note(int(note))
                chord_notes.append(new_note)
            new_chord = music21.chord.Chord(chord_notes)
            output_notes.append(new_chord)
        else:
            new_note = music21.note.Note(pattern)
            output_notes.append(new_note)
    
    midi_stream = music21.stream.Stream(output_notes)
    midi_stream.write("midi", filename)
    print("Music saved as", filename)

save_midi(generated_notes)


Music saved as generated_music.mid



# CONCLUSION

### This AI system:
- Learns patterns from MIDI files
- Uses Markov probability transitions
 - Generates new music compositions
 - Saves output as a playable MIDI file

#### Improvements:
- Increase dataset size
- Train longer epochs
 - Deploy via web app

##### "It demostrates probalistic sequence learning for symbolic music generation under low resource constraints."